In [1]:
import pandas as pd
import glob2
import matplotlib.pyplot as plt
import datetime
import re
from datetime import datetime, timedelta
%matplotlib inline

In [2]:
global_music_file_paths = glob2.glob('spotify korea v2/*.csv')

In [3]:
print(len(global_music_file_paths))

177


In [4]:
print(global_music_file_paths[:4])

['spotify korea v2\\regional-kr-weekly-2021-02-04.csv', 'spotify korea v2\\regional-kr-weekly-2021-02-11.csv', 'spotify korea v2\\regional-kr-weekly-2021-02-18.csv', 'spotify korea v2\\regional-kr-weekly-2021-02-25.csv']


In [5]:
# global_music_file_paths

#### Add Dates to the Files

In [6]:
#Add the dates to the files - run once
def addsDatesToData():
    to_remove_front = "spotify korea v2\\regional-kr-weekly-"
    to_remove_back = ".csv"

    len_front_remove = len(to_remove_front)
    len_back_remove = len(to_remove_back)

    for file in global_music_file_paths: 
#         print(file)
        the_data_in_file = pd.read_csv(file)
#         the_data_in_file = the_data_in_file.iloc[1: , :]
#         the_data_in_file.columns = ["Position", "Track Name", "Artist", "Streams", "URL"]
        
        
        the_dates = file[len_front_remove:]
        the_dates = the_dates[:-len_back_remove]

        split_dates = the_dates.split("-")

        start_date = split_dates[0] + "-" + split_dates[1] + "-" + split_dates[2]
        

        len_dates = the_data_in_file.shape[0]
        start_dates = len_dates * [start_date]

        date_columns = {"End Date": start_dates}   

        the_data_in_file["End Date"] = start_dates

        #write the data back to the File
        the_data_in_file.to_csv(file, index=False)
        
#remove dates in existing files
def removeExistingDates():
    for file in global_music_file_paths:
        
        the_data_in_file = pd.read_csv(file)

        the_data_in_file.drop(["Start Date", "End Date"], axis=1, inplace=True) 

        the_data_in_file.to_csv(file, index=False)
    
#check for dates ... opting for removal and putting them back on


In [7]:
# removeExistingDates()
addsDatesToData()

In [8]:
#Combine the files and sort by dates
all_files = []
trouble_files = []
for file in global_music_file_paths:
    the_data_in_file = pd.read_csv(file)
    
    if("URL" in list(the_data_in_file.columns)):
    
        the_data_in_file = the_data_in_file.drop(["URL"], axis=1)
    
    list_vals = sum(list(the_data_in_file.isnull().sum()))
    
    if(list_vals > 0):
        trouble_files.append(file)
    all_files.append(the_data_in_file)
    
the_spotify_data = pd.concat(all_files) 
# trouble_files_df = pd.concat(trouble_files)

In [9]:
all_cols = list(the_spotify_data.columns)
drop_cols = []

for col in all_cols:
    if("Unnamed" in col or "artificial" in col):
        drop_cols.append(col)
        
the_spotify_data.drop(drop_cols, axis=1, inplace = True)

In [10]:
the_spotify_data.columns

Index(['rank', 'uri', 'artist_names', 'track_name', 'source', 'peak_rank',
       'previous_rank', 'weeks_on_chart', 'streams', 'End Date'],
      dtype='object')

In [11]:
the_spotify_data

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,End Date
0,1,spotify:track:4saklk6nie3yiGePpBwUoc,BTS,Dynamite,BIGHIT MUSIC,1,-1,1,85091,2021-02-04
1,2,spotify:track:249gnXrbfmV8NG6jTEMSwD,BTS,Life Goes On,BIGHIT MUSIC,2,-1,1,73470,2021-02-04
2,3,spotify:track:4Ws314Ylb27BVsvlZOy30C,BLACKPINK,Lovesick Girls,YG Entertainment/Interscope Records,3,-1,1,42246,2021-02-04
3,4,spotify:track:0n2moJpAEWHwaPYYjkzMDl,BTS,Blue & Grey,BIGHIT MUSIC,4,-1,1,41002,2021-02-04
4,5,spotify:track:3QH8rQGNFX8VLbCgZ7uPTS,BTS,Fly To My Room,BIGHIT MUSIC,5,-1,1,37015,2021-02-04
...,...,...,...,...,...,...,...,...,...,...
195,196,spotify:track:3DO8WRX72c1Z5hduVL1Nd5,1415,선을 그어 주던가,Universal Music Ltd.,108,187,19,30834,2024-06-27
196,197,spotify:track:4cluDES4hQEUhmXj6TXkSo,One Direction,What Makes You Beautiful,Syco Music UK,182,188,5,30828,2024-06-27
197,198,spotify:track:1Y5D7m2QtyBdvwvY3gAVNQ,DOYOUNG,Little Light,SM Entertainment,43,197,10,30758,2024-06-27
198,199,spotify:track:1YsU8rW2u8z4F0pwOBQ4Ea,Coldplay,feelslikeimfallinginlove,Parlophone UK,199,-1,1,30660,2024-06-27


In [12]:
# the_spotify_data["End Date"].unique().tolist()

### Error checking that all the Thursday dates are present

In [13]:
date_list = the_spotify_data["End Date"].unique().tolist()
start_date = date_list[0]
end_date = date_list[len(date_list)-1]

In [14]:
len(date_list)

177

In [15]:
start_date, end_date

('2021-02-04', '2024-06-27')

In [16]:
def check_thursdays_in_range(date_list, start_date, end_date):
    # Convert start_date and end_date strings to datetime objects
    start_date = datetime.strptime(start_date, "%Y-%m-%d")
    end_date = datetime.strptime(end_date, "%Y-%m-%d")
    
    # Initialize a set to store Thursdays in the range
    thursdays_in_range = set()
    
    # Iterate through the range of dates and add Thursdays to the set
    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() == 3:  # Thursday
            thursdays_in_range.add(current_date.strftime("%Y-%m-%d"))
        current_date += timedelta(days=1)
    
    # Convert date_list to a set for efficient membership checking
    date_set = set(date_list)
    
    # Find missing Thursdays
    missing_thursdays = sorted(thursdays_in_range - date_set)
    print("thursdays_in_range: ", len(thursdays_in_range))
    print("date_set: ", len(date_set))
    
    if missing_thursdays:
        print("Missing Thursdays in the range:")
        for thursday in missing_thursdays:
            print(thursday)
        return False
    else:
        print("All Thursdays are present in the range.")
        return True

In [17]:
result = check_thursdays_in_range(date_list, start_date, end_date)
print("All Thursdays present:", result)

thursdays_in_range:  178
date_set:  177
Missing Thursdays in the range:
2024-06-20
All Thursdays present: False


### Continue

In [18]:
the_spotify_data["Week"] = the_spotify_data["End Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
the_spotify_data["Year"] = the_spotify_data["Week"]

the_spotify_data['Week'] = the_spotify_data['Week'].apply(lambda x : x.isocalendar()[1] )
the_spotify_data['Year'] = the_spotify_data['Year'].apply(lambda x : x.isocalendar()[0] )

# the_spotify_data["Start Date Dt"] = the_spotify_data["Start Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
the_spotify_data["End Date Dt"] = the_spotify_data["End Date"].apply(lambda x : datetime.strptime(x, '%Y-%m-%d'))
the_spotify_data['Seconds since Epoch'] = the_spotify_data['End Date Dt'].apply(lambda x : round(x.timestamp(), 0))

In [19]:
#get the month when this happened
def changedaytotheFirst(x):
    x = x[:-2] + "01"
    
    return x

In [20]:
the_spotify_data["End Date Aggregated"] = the_spotify_data["End Date"].apply(lambda x : changedaytotheFirst(x) )

In [21]:
#find the number of artists on the track
the_spotify_data.dropna(inplace=True)

the_spotify_data["artist_names"] = the_spotify_data["artist_names"].apply(lambda x : x.encode("utf-8").decode("latin-1"))
the_spotify_data["artist_names"] = the_spotify_data["artist_names"].apply(lambda x : re.sub(r'[^\x00-\x7F]+','', x))
the_spotify_data["artist_names"] = the_spotify_data.apply(lambda x : x["artist_names"] + "Artist_NA" if(x["artist_names"] == "") else x["artist_names"], axis=1)

the_spotify_data["track_name"] = the_spotify_data["track_name"].apply(lambda x : x.encode("utf-8").decode("latin-1"))
the_spotify_data["track_name"] = the_spotify_data["track_name"].apply(lambda x : re.sub(r'[^\x00-\x7F]+','', x))
the_spotify_data["track_name"] = the_spotify_data.apply(lambda x : x["artist_names"] + "_NA" if(x["track_name"] == "") else x["track_name"], axis=1)

def countNumberArtists(x):
    len_all_featured_artists = 0
    num_artists = 0
    xvals = x.split("feat")
    len_xvals = len(xvals)
    
    if(len_xvals > 1):
        featured_artists = xvals[1:]
        featured_artists_str = ' '.join(featured_artists)
        all_featured_artists = featured_artists_str.split("& ")
        len_all_featured_artists = len(all_featured_artists)
        
    num_artists = 1 + len_all_featured_artists
    
    return num_artists
        
    
the_spotify_data['ArtistCount'] = the_spotify_data['track_name'].apply(lambda x : countNumberArtists(x))

In [22]:
the_spotify_data.shape

(35400, 16)

In [23]:
the_spotify_data["main_artist"] = the_spotify_data.apply(lambda x : x["artist_names"].split(",")[0], axis=1)

In [24]:
the_spotify_data = the_spotify_data.reset_index()
the_spotify_data.drop(["index"], axis=1, inplace=True)

In [25]:
the_spotify_data.shape

(35400, 17)

### Is Top Ten Status

In [26]:
def everInTopTen(x):
    position_vals = list(x["rank"])
    position_vals = list(set(position_vals))
    
#     print(position_vals)
    
    min_pos = min(position_vals)
    existence_val = 0
    
    if(min_pos <= 10):
        existence_val = 1
    
    x["isTopTen"] = [existence_val] * len(x)
    
    return x
    
the_spotify_data = the_spotify_data[~the_spotify_data.index.duplicated()]
the_spotify_data = the_spotify_data.groupby(["artist_names", "track_name"]).apply(lambda x : everInTopTen(x))

In [27]:
the_spotify_data.head()

,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,End Date,Week,Year,End Date Dt,Seconds since Epoch,End Date Aggregated,ArtistCount,main_artist,isTopTen
0,1,spotify:track:4saklk6nie3yiGePpBwUoc,BTS,Dynamite,BIGHIT MUSIC,1,-1,1,85091,2021-02-04,5,2021,2021-02-04,1.612397e+09,2021-02-01,1,BTS,1
1,2,spotify:track:249gnXrbfmV8NG6jTEMSwD,BTS,Life Goes On,BIGHIT MUSIC,2,-1,1,73470,2021-02-04,5,2021,2021-02-04,1.612397e+09,2021-02-01,1,BTS,1
2,3,spotify:track:4Ws314Ylb27BVsvlZOy30C,BLACKPINK,Lovesick Girls,YG Entertainment/Interscope Records,3,-1,1,42246,2021-02-04,5,2021,2021-02-04,1.612397e+09,2021-02-01,1,BLACKPINK,1
3,4,spotify:track:0n2moJpAEWHwaPYYjkzMDl,BTS,Blue & Grey,BIGHIT MUSIC,4,-1,1,41002,2021-02-04,5,2021,2021-02-04,1.612397e+09,2021-02-01,1,BTS,1
4,5,spotify:track:3QH8rQGNFX8VLbCgZ7uPTS,BTS,Fly To My Room,BIGHIT MUSIC,5,-1,1,37015,2021-02-04,5,2021,2021-02-04,1.612397e+09,2021-02-01,1,BTS,1


### When did the track reach the Top 10 

### how long does a song last on the charts?

In [28]:
the_spotify_data["isTopTen"].value_counts()

0    24080
1    11320
Name: isTopTen, dtype: int64

In [29]:
the_spotify_data.isnull().sum()

rank                   0
uri                    0
artist_names           0
track_name             0
source                 0
peak_rank              0
previous_rank          0
weeks_on_chart         0
streams                0
End Date               0
Week                   0
Year                   0
End Date Dt            0
Seconds since Epoch    0
End Date Aggregated    0
ArtistCount            0
main_artist            0
isTopTen               0
dtype: int64

In [30]:
unique_artists = the_spotify_data["main_artist"].unique().tolist()
num_artists = len(unique_artists)
print(num_artists)

701


### find new artists in that week

In [31]:
# the_spotify_data.head(12)
def getArtistAppearanceCount(x):
    
    x = x.reset_index()
    x.drop(["index"],axis=1,inplace=True)
    x = x.reset_index()
    
    x_cols = x.columns
    
    x_cols = [w.replace('index', 'artistAppearanceCount') for w in x_cols]
    
    x.columns = x_cols
    
    return x        
    
the_spotify_data =the_spotify_data.groupby(["main_artist"], as_index=False).apply(lambda x : getArtistAppearanceCount(x))
the_spotify_data = the_spotify_data.reset_index()
the_spotify_data.drop(["level_0", "level_1"], inplace=True, axis=1)

In [32]:
the_spotify_data.head()

,artistAppearanceCount,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,End Date,Week,Year,End Date Dt,Seconds since Epoch,End Date Aggregated,ArtistCount,main_artist,isTopTen
0,0,181,spotify:track:2bU2EvhMM4Z1M1CLJlnZH1,Damons year,yours,Akashic Records,181,-1,1,16016,2022-08-25,34,2022,2022-08-25,1.661386e+09,2022-08-01,1,Damons year,0
1,1,161,spotify:track:2bU2EvhMM4Z1M1CLJlnZH1,Damons year,yours,Akashic Records,161,181,2,17530,2022-09-01,35,2022,2022-09-01,1.661990e+09,2022-09-01,1,Damons year,0
2,2,121,spotify:track:2bU2EvhMM4Z1M1CLJlnZH1,Damons year,yours,Akashic Records,121,161,3,21163,2022-09-08,36,2022,2022-09-08,1.662595e+09,2022-09-01,1,Damons year,0
3,3,147,spotify:track:2bU2EvhMM4Z1M1CLJlnZH1,Damons year,yours,Akashic Records,121,121,4,18562,2022-09-15,37,2022,2022-09-15,1.663200e+09,2022-09-01,1,Damons year,0
4,4,179,spotify:track:2bU2EvhMM4Z1M1CLJlnZH1,Damons year,yours,Akashic Records,121,147,5,18780,2022-09-22,38,2022,2022-09-22,1.663805e+09,2022-09-01,1,Damons year,0


#### Track Appearance Count

In [33]:
def getTrackAppearanceCount(x):
    
    x = x.reset_index()
    x.drop(["index"],axis=1,inplace=True)
    x = x.reset_index()
    
    x_cols = x.columns
    
    x_cols = [w.replace('index', 'trackAppearanceCount') for w in x_cols]
    
    x.columns = x_cols
    
    return x         
    
the_spotify_data =the_spotify_data.groupby(["artist_names", "track_name"], as_index=False).apply(lambda x : getTrackAppearanceCount(x))
the_spotify_data = the_spotify_data.reset_index()
the_spotify_data.drop(["level_0", "level_1"], inplace=True, axis=1)

In [34]:
the_spotify_data.head()

,trackAppearanceCount,artistAppearanceCount,rank,uri,artist_names,track_name,source,peak_rank,previous_rank,weeks_on_chart,streams,End Date,Week,Year,End Date Dt,Seconds since Epoch,End Date Aggregated,ArtistCount,main_artist,isTopTen
0,0,20,163,spotify:track:1bUacg9mWo407qZvsknlFd,Damons year,josee!,Akashic Records,163,-1,1,22086,2023-01-05,1,2023,2023-01-05,1.672877e+09,2023-01-01,1,Damons year,0
1,1,22,149,spotify:track:1bUacg9mWo407qZvsknlFd,Damons year,josee!,Akashic Records,149,163,2,22576,2023-01-12,2,2023,2023-01-12,1.673482e+09,2023-01-01,1,Damons year,0
2,2,24,102,spotify:track:1bUacg9mWo407qZvsknlFd,Damons year,josee!,Akashic Records,102,149,3,27953,2023-01-19,3,2023,2023-01-19,1.674086e+09,2023-01-01,1,Damons year,0
3,3,26,191,spotify:track:1bUacg9mWo407qZvsknlFd,Damons year,josee!,Akashic Records,102,102,4,18211,2023-01-26,4,2023,2023-01-26,1.674691e+09,2023-01-01,1,Damons year,0
4,4,28,150,spotify:track:1bUacg9mWo407qZvsknlFd,Damons year,josee!,Akashic Records,102,191,5,22987,2023-02-02,5,2023,2023-02-02,1.675296e+09,2023-02-01,1,Damons year,0


In [35]:
the_spotify_data.to_csv("the_spotify_data_kr.csv")

In [36]:
the_spotify_data["rank"] = the_spotify_data["rank"].astype(int)
the_spotify_data["streams"] = the_spotify_data["streams"].astype(int)
the_spotify_data["ArtistCount"] = the_spotify_data["ArtistCount"].astype(int)

the_spotify_data["isTopTen"] = the_spotify_data["isTopTen"].astype(int)
# the_spotify_data["lenOnCharts"] = the_spotify_data["lenOnCharts"].astype(int)
the_spotify_data["Year"] = the_spotify_data["Year"].astype(int)
the_spotify_data["Week"] = the_spotify_data["Week"].astype(int)

In [37]:
all_cols = list(the_spotify_data.columns)
drop_cols = []

for col in all_cols:
    if("Unnamed" in col or "artificial" in col):
        drop_cols.append(col)
        
the_spotify_data.drop(drop_cols, axis=1, inplace = True)

In [38]:
the_spotify_data["Artist and Track"] = the_spotify_data["artist_names"] + "; " + the_spotify_data["track_name"]

### Getting the list of Artists and Tracks

In [39]:
import pandas as pd
# import pymongo
# import getlyrics

In [40]:
def trackAppearance(x):
#     x = x.reset_index()

    x = x.sort_values(by=["End Date"])
#     x.set_index(["Position"])

    x["rank"] = x["rank"].astype(int)
#     print(list(x["Position"]))
    
    x["rank difference"] = x["rank"].diff().fillna(0)
    x["rank difference"] = x["rank difference"].astype(int)
    
#     x["Position Difference"] = x["Position"].sub(x["Position"].shift())
#     x.drop(["index"], inplace=True)
    
    return x
    
all_files_grouped = the_spotify_data.groupby(["main_artist", "track_name"]).apply(lambda x: trackAppearance(x))

In [41]:
all_files_grouped["rank difference"].max()

185

### Position over time

In [42]:
def positionvertime(x):
    posovertime = 0
    #new track
    if(x["rank difference"] == 0 and x["trackAppearanceCount"]==0):
        posovertime = 0
    #track stayed in the same position
    elif(x["rank difference"] == 0 and x["trackAppearanceCount"]!=0):
        posovertime = 50
    #track went up the chart
    elif(x["rank difference"] < 0):
        posovertime = 75
    #track fell down the chart
    else:
        posovertime = 100
    return posovertime

In [43]:
all_files_grouped["Position over Time"] = all_files_grouped.apply(lambda x : positionvertime(x), axis=1)

##### Add a Girl Group and Boy Group

In [44]:
ggdf = pd.read_csv("Classification/ListofGirlGroups.csv", encoding='ANSI')
bgdf = pd.read_csv("Classification/ListofBoyGroups.csv", on_bad_lines='skip', encoding='ANSI')

In [45]:
def IsGG(ggdf, y):
    ggs = [x.lower().strip() for x in ggdf["Artist"]]
    y = y.lower().strip()
    
    if y in ggs:
        return True
    else:
        return False

def IsBG(bgdf, y):
    bgs = [x.lower().strip() for x in bgdf["Artist"]]
    y = y.lower().strip()
    
    if y in bgs:
        return True
    else:
        return False

In [46]:
all_files_grouped["IsGirlGroup"] = all_files_grouped["main_artist"].apply(lambda x : IsGG(ggdf, x))
all_files_grouped["IsBoyGroup"] = all_files_grouped["main_artist"].apply(lambda x : IsGG(bgdf, x))

In [47]:
ggs = all_files_grouped[all_files_grouped["IsGirlGroup"]==True]["main_artist"].unique().tolist()
bgs = all_files_grouped[all_files_grouped["IsBoyGroup"]==True]["main_artist"].unique().tolist()

In [48]:
all_files_grouped.to_csv("Classification/all_files_SKRv1.csv", index=False)

In [49]:
all_files_grouped.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'rank', 'uri',
       'artist_names', 'track_name', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time', 'IsGirlGroup', 'IsBoyGroup'],
      dtype='object')

In [50]:
# new_all_files_grouped = all_files_grouped.reset_index(drop=True)

In [51]:
def newstuff(x):
#     x = x.reset_index()

    x = x.sort_values(by=['Seconds since Epoch'])
#     x.set_index(["Position"])
    print(x)
    
    return x
    


In [52]:
# all_files_grouped = new_all_files_grouped.groupby(["Artist", "Track Name"]).apply(lambda x: newstuff(x))

In [53]:
# all_files_grouped.min()

In [54]:
# df = pd.DataFrame({'A': 'a a b'.split(), 'B': [1,2,3], 'C': [4,6, 5]})

In [55]:
# new_all_files_grouped.groupby([])

In [56]:
# all_files_grouped.apply(lambda x : print(type(x)) )

In [57]:
# # new_all_files_grouped.h'Track Name', 'Artist'ead(5)
# cols_ = ['Start Date', 'Seconds since Epoch', 'Position']
# new_test_df = pd.DataFrame()

In [58]:
# 604800
def weekonweekposition(x):
    
#     print(x.values)
    
    pd.DataFrame({'email':x.index, 'list':x.values})
    
    
#     return x_
    
#     x_ = pd.DataFrame(x, columns=['Start Date'])
    
#     x_ = x.to_frame()
    
#     print(x_.columns)
    
#     for n in range(0, fin_len):
#         for col in cols_:
#             print(len(x[n]), x[n], col)
        
    
#     x_ = pd.DataFrame(x, columns=cols_)
#     print(x_.shape)
#     print(x.columns)

In [59]:
# all_files_gr = new_all_files_grouped[['Artist', 'Track Name', 'Seconds since Epoch', 'Position']]

In [60]:
# all_files_grouped[cols_].apply( lambda x : weekonweekposition(x) )

In [61]:
# new_test_df

### South African Artists

In [62]:
all_files_grouped.shape

(35400, 25)

In [63]:
all_files_grouped.columns

Index(['trackAppearanceCount', 'artistAppearanceCount', 'rank', 'uri',
       'artist_names', 'track_name', 'source', 'peak_rank', 'previous_rank',
       'weeks_on_chart', 'streams', 'End Date', 'Week', 'Year', 'End Date Dt',
       'Seconds since Epoch', 'End Date Aggregated', 'ArtistCount',
       'main_artist', 'isTopTen', 'Artist and Track', 'rank difference',
       'Position over Time', 'IsGirlGroup', 'IsBoyGroup'],
      dtype='object')

In [64]:
narrowing_df = all_files_grouped[['End Date', "rank difference"]]

In [65]:
narrowing_df.reset_index(inplace=True)

In [66]:
narrowing_df.drop(["level_2"], axis=1, inplace=True)

C:\Users\User\AppData\Local\Temp\ipykernel_21124\3213521647.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  narrowing_df.drop(["level_2"], axis=1, inplace=True)


In [67]:
narrowing_df = narrowing_df[narrowing_df["rank difference"] == 0]

In [68]:
narrowing_df.to_csv("narrow_uniqkr.csv", index=False)